In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import sqlite3
con = sqlite3.connect('../summer')
cur = con.cursor()

In [2]:
students = ['\'Alonzo\'','\'Dax\'','\'Emily\'','\'Isabelle\'','\'Jake\'','\'Philip\'','\'Rishab\'','\'Sam\'',
            '\'justin\'','\'Alex\'','\'Jordan\'']
recordings = ['\'week1_3\'', '\'week2_3\'', '\'week3_3\'', '\'week4_3\'', '\'week5_3\'', '\'week6_3\'', '\'week7_3\'', '\'week8_3\'']

all_weeks = pd.read_sql_query("""SELECT PlayerName, RecordingName,
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName IN ({})
    AND RecordingName IN ({})
    """.format(",".join(students), ",".join(recordings)), con)

print(all_weeks.shape)

(4109420, 23)


In [9]:
translations = ['HeadPos_x', 'HeadPos_y', 'HeadPos_z',
                'LeftHandPos_x', 'LeftHandPos_y', 'LeftHandPos_z',
                'RightHandPos_x', 'RightHandPos_y', 'RightHandPos_z']

playernames = all_weeks['PlayerName'].unique()
recordingnames = all_weeks['RecordingName'].unique()

playerIdx = {
    'Alonzo': 0,
    'Dax': 1,
    'Emily': 2,
    'Isabelle': 3,
    'Jake': 4,
    'Philip': 5,
    'Rishab': 6,
    'Sam': 7,
    'justin': 8,
    'Alex': 9,
    'Jordan': 10
}

X_train = []
y_train = []
X_test = []
y_test = []

for recording_name in recordingnames:
    recording_df = all_weeks[all_weeks['RecordingName'] == recording_name]

    for name in playernames:
        player_df = recording_df[recording_df['PlayerName'] == name]
        
        training_frames = math.ceil(player_df.shape[0] * 0.8)

        for i in range(0, player_df.shape[0] - 29, 30):
            features = []
            window = player_df.iloc[i:i+29]

            for column_name in translations:
                column = window[column_name]
                features.extend([column.max(), column.min(), column.median(), column.mean(), column.std()])
                
            head_alpha, lhand_alpha, rhand_alpha = [], [], []
            head_beta, lhand_beta, rhand_beta = [], [], []
            head_gamma, lhand_gamma, rhand_gamma = [], [], []
            for j in range(i, i+29):
                head_q0, head_q1, head_q2, head_q3 = player_df.iloc[j]['HeadRot_w'], player_df.iloc[j]['HeadRot_x'], player_df.iloc[j]['HeadRot_y'], player_df.iloc[j]['HeadRot_z'], 
                lhand_q0, lhand_q1, lhand_q2, lhand_q3 = player_df.iloc[j]['LeftHandRot_w'], player_df.iloc[j]['LeftHandRot_x'], player_df.iloc[j]['LeftHandRot_y'], player_df.iloc[j]['LeftHandRot_z'], 
                rhand_q0, rhand_q1, rhand_q2, rhand_q3 = player_df.iloc[j]['RightHandRot_w'], player_df.iloc[j]['RightHandRot_x'], player_df.iloc[j]['RightHandRot_y'], player_df.iloc[j]['RightHandRot_z'], 
                
                head_alpha.append(math.atan2(2*(head_q0*head_q1+head_q2*head_q3), 1-2*(head_q1**2+head_q2**2)))
                head_beta.append(math.asin(2*(head_q0*head_q2-head_q3*head_q1)))
                head_gamma.append(math.atan2(2*(head_q0*head_q3+head_q1*head_q2), 1-2*(head_q2**2+head_q3**2)))
                
                lhand_alpha.append(math.atan2(2*(lhand_q0*lhand_q1+lhand_q2*lhand_q3), 1-2*(lhand_q1**2+lhand_q2**2)))
                lhand_beta.append(math.asin(2*(lhand_q0*lhand_q2-lhand_q3*lhand_q1)))
                lhand_gamma.append(math.atan2(2*(lhand_q0*lhand_q3+lhand_q1*lhand_q2), 1-2*(lhand_q2**2+lhand_q3**2)))
                
                rhand_alpha.append(math.atan2(2*(rhand_q0*rhand_q1+rhand_q2*rhand_q3), 1-2*(rhand_q1**2+rhand_q2**2)))
                rhand_beta.append(math.asin(2*(rhand_q0*rhand_q2-rhand_q3*rhand_q1)))
                rhand_gamma.append(math.atan2(2*(rhand_q0*rhand_q3+rhand_q1*rhand_q2), 1-2*(rhand_q2**2+rhand_q3**2)))
                
            features.extend([max(head_alpha), min(head_alpha), np.median(head_alpha), np.mean(head_alpha), np.std(head_alpha)])
            features.extend([max(head_beta), min(head_beta), np.median(head_beta), np.mean(head_beta), np.std(head_beta)])
            features.extend([max(head_gamma), min(head_gamma), np.median(head_gamma), np.mean(head_gamma), np.std(head_gamma)])
            
            features.extend([max(lhand_alpha), min(lhand_alpha), np.median(lhand_alpha), np.mean(lhand_alpha), np.std(lhand_alpha)])
            features.extend([max(lhand_beta), min(lhand_beta), np.median(lhand_beta), np.mean(lhand_beta), np.std(lhand_beta)])
            features.extend([max(lhand_gamma), min(lhand_gamma), np.median(lhand_gamma), np.mean(lhand_gamma), np.std(lhand_gamma)])
            
            features.extend([max(rhand_alpha), min(rhand_alpha), np.median(rhand_alpha), np.mean(rhand_alpha), np.std(rhand_alpha)])
            features.extend([max(rhand_beta), min(rhand_beta), np.median(rhand_beta), np.mean(rhand_beta), np.std(rhand_beta)])
            features.extend([max(rhand_gamma), min(rhand_gamma), np.median(rhand_gamma), np.mean(rhand_gamma), np.std(rhand_gamma)])
            
            if i < training_frames:
                X_train.append(features)
                y_train.append(playerIdx[name])
                
            else:
                X_test.append(features)
                y_test.append(playerIdx[name])
            

print(len(X_train))
print(len(X_train[0]))
print(len(y_train))
print(len(X_test))
print(len(X_test[0]))
print(len(y_test))

109620
90
109620
27322
90
27322


In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=42)

In [12]:
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7577044140253276


baseline paper random forest - 0.753683056114056
with euler angles - 0.7577044140253276

In [13]:
X_train_3s = []
y_train_3s = []
X_test_3s = []
y_test_3s = []

for recording_name in recordingnames:
    recording_df = all_weeks[all_weeks['RecordingName'] == recording_name]

    for name in playernames:
        player_df = recording_df[recording_df['PlayerName'] == name]
        
        training_frames = math.ceil(player_df.shape[0] * 0.8)

        for i in range(0, player_df.shape[0] - 89, 90):
            features = []
            window = player_df.iloc[i:i+89]

            for column_name in translations:
                column = window[column_name]
                features.extend([column.max(), column.min(), column.median(), column.mean(), column.std()])
                
            head_alpha, lhand_alpha, rhand_alpha = [], [], []
            head_beta, lhand_beta, rhand_beta = [], [], []
            head_gamma, lhand_gamma, rhand_gamma = [], [], []
            for j in range(i, i+89):
                head_q0, head_q1, head_q2, head_q3 = player_df.iloc[j]['HeadRot_w'], player_df.iloc[j]['HeadRot_x'], player_df.iloc[j]['HeadRot_y'], player_df.iloc[j]['HeadRot_z'], 
                lhand_q0, lhand_q1, lhand_q2, lhand_q3 = player_df.iloc[j]['LeftHandRot_w'], player_df.iloc[j]['LeftHandRot_x'], player_df.iloc[j]['LeftHandRot_y'], player_df.iloc[j]['LeftHandRot_z'], 
                rhand_q0, rhand_q1, rhand_q2, rhand_q3 = player_df.iloc[j]['RightHandRot_w'], player_df.iloc[j]['RightHandRot_x'], player_df.iloc[j]['RightHandRot_y'], player_df.iloc[j]['RightHandRot_z'], 
                
                head_alpha.append(math.atan2(2*(head_q0*head_q1+head_q2*head_q3), 1-2*(head_q1**2+head_q2**2)))
                head_beta.append(math.asin(2*(head_q0*head_q2-head_q3*head_q1)))
                head_gamma.append(math.atan2(2*(head_q0*head_q3+head_q1*head_q2), 1-2*(head_q2**2+head_q3**2)))
                
                lhand_alpha.append(math.atan2(2*(lhand_q0*lhand_q1+lhand_q2*lhand_q3), 1-2*(lhand_q1**2+lhand_q2**2)))
                lhand_beta.append(math.asin(2*(lhand_q0*lhand_q2-lhand_q3*lhand_q1)))
                lhand_gamma.append(math.atan2(2*(lhand_q0*lhand_q3+lhand_q1*lhand_q2), 1-2*(lhand_q2**2+lhand_q3**2)))
                
                rhand_alpha.append(math.atan2(2*(rhand_q0*rhand_q1+rhand_q2*rhand_q3), 1-2*(rhand_q1**2+rhand_q2**2)))
                rhand_beta.append(math.asin(2*(rhand_q0*rhand_q2-rhand_q3*rhand_q1)))
                rhand_gamma.append(math.atan2(2*(rhand_q0*rhand_q3+rhand_q1*rhand_q2), 1-2*(rhand_q2**2+rhand_q3**2)))
                
            features.extend([max(head_alpha), min(head_alpha), np.median(head_alpha), np.mean(head_alpha), np.std(head_alpha)])
            features.extend([max(head_beta), min(head_beta), np.median(head_beta), np.mean(head_beta), np.std(head_beta)])
            features.extend([max(head_gamma), min(head_gamma), np.median(head_gamma), np.mean(head_gamma), np.std(head_gamma)])
            
            features.extend([max(lhand_alpha), min(lhand_alpha), np.median(lhand_alpha), np.mean(lhand_alpha), np.std(lhand_alpha)])
            features.extend([max(lhand_beta), min(lhand_beta), np.median(lhand_beta), np.mean(lhand_beta), np.std(lhand_beta)])
            features.extend([max(lhand_gamma), min(lhand_gamma), np.median(lhand_gamma), np.mean(lhand_gamma), np.std(lhand_gamma)])
            
            features.extend([max(rhand_alpha), min(rhand_alpha), np.median(rhand_alpha), np.mean(rhand_alpha), np.std(rhand_alpha)])
            features.extend([max(rhand_beta), min(rhand_beta), np.median(rhand_beta), np.mean(rhand_beta), np.std(rhand_beta)])
            features.extend([max(rhand_gamma), min(rhand_gamma), np.median(rhand_gamma), np.mean(rhand_gamma), np.std(rhand_gamma)])
            
            if i < training_frames:
                X_train_3s.append(features)
                y_train_3s.append(playerIdx[name])
                
            else:
                X_test_3s.append(features)
                y_test_3s.append(playerIdx[name])
            

print(len(X_train_3s))
print(len(X_train_3s[0]))
print(len(y_train_3s))
print(len(X_test_3s))
print(len(X_test_3s[0]))
print(len(y_test_3s))

36565
90
36565
9055
90
9055


In [14]:
X_train_3s = scaler.fit_transform(X_train_3s)
X_test_3s = scaler.transform(X_test_3s)
classifier_3s = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier_3s.fit(X_train_3s, y_train_3s)
y_pred_3s = classifier_3s.predict(X_test_3s)
print(accuracy_score(y_test_3s, y_pred_3s))

0.7499723909442297


In [15]:
X_train_10s = []
y_train_10s = []
X_test_10s = []
y_test_10s = []

for recording_name in recordingnames:
    recording_df = all_weeks[all_weeks['RecordingName'] == recording_name]

    for name in playernames:
        player_df = recording_df[recording_df['PlayerName'] == name]
        
        training_frames = math.ceil(player_df.shape[0] * 0.8)

        for i in range(0, player_df.shape[0] - 299, 300):
            features = []
            window = player_df.iloc[i:i+299]

            for column_name in translations:
                column = window[column_name]
                features.extend([column.max(), column.min(), column.median(), column.mean(), column.std()])
                
            head_alpha, lhand_alpha, rhand_alpha = [], [], []
            head_beta, lhand_beta, rhand_beta = [], [], []
            head_gamma, lhand_gamma, rhand_gamma = [], [], []
            for j in range(i, i+299):
                head_q0, head_q1, head_q2, head_q3 = player_df.iloc[j]['HeadRot_w'], player_df.iloc[j]['HeadRot_x'], player_df.iloc[j]['HeadRot_y'], player_df.iloc[j]['HeadRot_z'], 
                lhand_q0, lhand_q1, lhand_q2, lhand_q3 = player_df.iloc[j]['LeftHandRot_w'], player_df.iloc[j]['LeftHandRot_x'], player_df.iloc[j]['LeftHandRot_y'], player_df.iloc[j]['LeftHandRot_z'], 
                rhand_q0, rhand_q1, rhand_q2, rhand_q3 = player_df.iloc[j]['RightHandRot_w'], player_df.iloc[j]['RightHandRot_x'], player_df.iloc[j]['RightHandRot_y'], player_df.iloc[j]['RightHandRot_z'], 
                
                head_alpha.append(math.atan2(2*(head_q0*head_q1+head_q2*head_q3), 1-2*(head_q1**2+head_q2**2)))
                head_beta.append(math.asin(2*(head_q0*head_q2-head_q3*head_q1)))
                head_gamma.append(math.atan2(2*(head_q0*head_q3+head_q1*head_q2), 1-2*(head_q2**2+head_q3**2)))
                
                lhand_alpha.append(math.atan2(2*(lhand_q0*lhand_q1+lhand_q2*lhand_q3), 1-2*(lhand_q1**2+lhand_q2**2)))
                lhand_beta.append(math.asin(2*(lhand_q0*lhand_q2-lhand_q3*lhand_q1)))
                lhand_gamma.append(math.atan2(2*(lhand_q0*lhand_q3+lhand_q1*lhand_q2), 1-2*(lhand_q2**2+lhand_q3**2)))
                
                rhand_alpha.append(math.atan2(2*(rhand_q0*rhand_q1+rhand_q2*rhand_q3), 1-2*(rhand_q1**2+rhand_q2**2)))
                rhand_beta.append(math.asin(2*(rhand_q0*rhand_q2-rhand_q3*rhand_q1)))
                rhand_gamma.append(math.atan2(2*(rhand_q0*rhand_q3+rhand_q1*rhand_q2), 1-2*(rhand_q2**2+rhand_q3**2)))
                
            features.extend([max(head_alpha), min(head_alpha), np.median(head_alpha), np.mean(head_alpha), np.std(head_alpha)])
            features.extend([max(head_beta), min(head_beta), np.median(head_beta), np.mean(head_beta), np.std(head_beta)])
            features.extend([max(head_gamma), min(head_gamma), np.median(head_gamma), np.mean(head_gamma), np.std(head_gamma)])
            
            features.extend([max(lhand_alpha), min(lhand_alpha), np.median(lhand_alpha), np.mean(lhand_alpha), np.std(lhand_alpha)])
            features.extend([max(lhand_beta), min(lhand_beta), np.median(lhand_beta), np.mean(lhand_beta), np.std(lhand_beta)])
            features.extend([max(lhand_gamma), min(lhand_gamma), np.median(lhand_gamma), np.mean(lhand_gamma), np.std(lhand_gamma)])
            
            features.extend([max(rhand_alpha), min(rhand_alpha), np.median(rhand_alpha), np.mean(rhand_alpha), np.std(rhand_alpha)])
            features.extend([max(rhand_beta), min(rhand_beta), np.median(rhand_beta), np.mean(rhand_beta), np.std(rhand_beta)])
            features.extend([max(rhand_gamma), min(rhand_gamma), np.median(rhand_gamma), np.mean(rhand_gamma), np.std(rhand_gamma)])
            
            if i < training_frames:
                X_train_10s.append(features)
                y_train_10s.append(playerIdx[name])
                
            else:
                X_test_10s.append(features)
                y_test_10s.append(playerIdx[name])
            

print(len(X_train_10s))
print(len(X_train_10s[0]))
print(len(y_train_10s))
print(len(X_test_10s))
print(len(X_test_10s[0]))
print(len(y_test_10s))

10999
90
10999
2656
90
2656


In [16]:
X_train_10s = scaler.fit_transform(X_train_10s)
X_test_10s = scaler.transform(X_test_10s)
classifier_10s = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier_10s.fit(X_train_10s, y_train_10s)
y_pred_10s = classifier_10s.predict(X_test_10s)
print(accuracy_score(y_test_10s, y_pred_10s))

0.7695783132530121


In [17]:
X_train_30s = []
y_train_30s = []
X_test_30s = []
y_test_30s = []

for recording_name in recordingnames:
    recording_df = all_weeks[all_weeks['RecordingName'] == recording_name]

    for name in playernames:
        player_df = recording_df[recording_df['PlayerName'] == name]
        
        training_frames = math.ceil(player_df.shape[0] * 0.8)

        for i in range(0, player_df.shape[0] - 899, 30): # check the starting frames
            features = []
            window = player_df.iloc[i:i+899]

            for column_name in translations:
                column = window[column_name]
                features.extend([column.max(), column.min(), column.median(), column.mean(), column.std()])
                
            head_alpha, lhand_alpha, rhand_alpha = [], [], []
            head_beta, lhand_beta, rhand_beta = [], [], []
            head_gamma, lhand_gamma, rhand_gamma = [], [], []
            for j in range(i, i+899):
                head_q0, head_q1, head_q2, head_q3 = player_df.iloc[j]['HeadRot_w'], player_df.iloc[j]['HeadRot_x'], player_df.iloc[j]['HeadRot_y'], player_df.iloc[j]['HeadRot_z'], 
                lhand_q0, lhand_q1, lhand_q2, lhand_q3 = player_df.iloc[j]['LeftHandRot_w'], player_df.iloc[j]['LeftHandRot_x'], player_df.iloc[j]['LeftHandRot_y'], player_df.iloc[j]['LeftHandRot_z'], 
                rhand_q0, rhand_q1, rhand_q2, rhand_q3 = player_df.iloc[j]['RightHandRot_w'], player_df.iloc[j]['RightHandRot_x'], player_df.iloc[j]['RightHandRot_y'], player_df.iloc[j]['RightHandRot_z'], 
                
                head_alpha.append(math.atan2(2*(head_q0*head_q1+head_q2*head_q3), 1-2*(head_q1**2+head_q2**2)))
                head_beta.append(math.asin(2*(head_q0*head_q2-head_q3*head_q1)))
                head_gamma.append(math.atan2(2*(head_q0*head_q3+head_q1*head_q2), 1-2*(head_q2**2+head_q3**2)))
                
                lhand_alpha.append(math.atan2(2*(lhand_q0*lhand_q1+lhand_q2*lhand_q3), 1-2*(lhand_q1**2+lhand_q2**2)))
                lhand_beta.append(math.asin(2*(lhand_q0*lhand_q2-lhand_q3*lhand_q1)))
                lhand_gamma.append(math.atan2(2*(lhand_q0*lhand_q3+lhand_q1*lhand_q2), 1-2*(lhand_q2**2+lhand_q3**2)))
                
                rhand_alpha.append(math.atan2(2*(rhand_q0*rhand_q1+rhand_q2*rhand_q3), 1-2*(rhand_q1**2+rhand_q2**2)))
                rhand_beta.append(math.asin(2*(rhand_q0*rhand_q2-rhand_q3*rhand_q1)))
                rhand_gamma.append(math.atan2(2*(rhand_q0*rhand_q3+rhand_q1*rhand_q2), 1-2*(rhand_q2**2+rhand_q3**2)))
                
            features.extend([max(head_alpha), min(head_alpha), np.median(head_alpha), np.mean(head_alpha), np.std(head_alpha)])
            features.extend([max(head_beta), min(head_beta), np.median(head_beta), np.mean(head_beta), np.std(head_beta)])
            features.extend([max(head_gamma), min(head_gamma), np.median(head_gamma), np.mean(head_gamma), np.std(head_gamma)])
            
            features.extend([max(lhand_alpha), min(lhand_alpha), np.median(lhand_alpha), np.mean(lhand_alpha), np.std(lhand_alpha)])
            features.extend([max(lhand_beta), min(lhand_beta), np.median(lhand_beta), np.mean(lhand_beta), np.std(lhand_beta)])
            features.extend([max(lhand_gamma), min(lhand_gamma), np.median(lhand_gamma), np.mean(lhand_gamma), np.std(lhand_gamma)])
            
            features.extend([max(rhand_alpha), min(rhand_alpha), np.median(rhand_alpha), np.mean(rhand_alpha), np.std(rhand_alpha)])
            features.extend([max(rhand_beta), min(rhand_beta), np.median(rhand_beta), np.mean(rhand_beta), np.std(rhand_beta)])
            features.extend([max(rhand_gamma), min(rhand_gamma), np.median(rhand_gamma), np.mean(rhand_gamma), np.std(rhand_gamma)])
            
            if i < training_frames:
                X_train_30s.append(features)
                y_train_30s.append(playerIdx[name])
                
            else:
                X_test_30s.append(features)
                y_test_30s.append(playerIdx[name])
            

print(len(X_train_30s))
print(len(X_train_30s[0]))
print(len(y_train_30s))
print(len(X_test_30s))
print(len(X_test_30s[0]))
print(len(y_test_30s))

3694
90
3694
831
90
831


In [18]:
X_train_30s = scaler.fit_transform(X_train_30s)
X_test_30s = scaler.transform(X_test_30s)
classifier_30s = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier_30s.fit(X_train_30s, y_train_30s)
y_pred_30s = classifier_30s.predict(X_test_30s)
print(accuracy_score(y_test_30s, y_pred_30s))

0.7785800240673887


center the windows on a point
points are every 30 frames instead of for example 900 frames

9 histograms, one for each of the calculated euler angles

eq10 on website - rotations in 2D
use this eq to calculate surging and strafing rotations - z and x

calculate the avg position of x and z
subtract all x and z values from this avg position, then rotate using the eq

max, min, etc for surging and strafing - mean will be 0

1/3 of a second for velocity (10 frames regardless of window)

no7 expand beyond just median to diff quartiles